In [1]:
import numpy as np
from itertools import product
import sys
sys.path.append('/home/samuel.varga/projects/deep_learning/')
from deep_learning.training_utils import read_rotation_results, load_hp_opt_results

In [2]:
results_dir = '/work/samuel.varga/projects/2to6_hr_severe_wx/DEEP_LEARNING/results/'
target_column = 'any_severe__36km'

In [3]:
combo_list = []
for p_s, lrate, cs, i, loss in product([0.01, 0.1, 0.25], [0.0001, 0.001, 0.01, 0.1], ([2,1,2,1],[2,2,2,2],[2,3,2,3],[4,3,2,2]), (1,2,3,4), ['binary_crossentropy']):
    #U-net architectural parameters
    conv_filters=[n*i for n in [32,64,128,256]]
    max_pool=[2,2,2,2]
    conv_layers =[{'filters': f, 'kernel_size': (s), 'pool_size': (p), 'strides': (p)} if p > 1
                       else {'filters': f, 'kernel_size': (s), 'pool_size': None, 'strides': None}
                       for s, f, p, in zip(cs, conv_filters, max_pool)]
    args={'lrate':lrate, 'loss':loss,'activation_conv':'relu','activation_out':'sigmoid',
         'p_spatial_dropout':p_s, 'filters':conv_filters, 'size':cs, 'pool':max_pool, 'shape':(16,16), 'target_column':target_column, 'i':i}
    fbase = f"{args['target_column']}_Rot_*_{args['shape'][0]}_{args['shape'][1]}_lrate_{args['lrate']}_spatial_dropout_{args['p_spatial_dropout']}_i_{args['i']}_filters_{args['filters']}_size_{args['size']}_pool_{args['pool']}_loss_{args['loss']}_results.pkl"
    combo_list.append(fbase)
    

In [4]:
results = load_hp_opt_results(results_dir, combo_list)

In [5]:
hp_results={}
for k in results.keys():
    temp = np.array([x['predict_val_eval'] for x in results[k]])
    temp = np.mean(temp, axis=0)
    hp_results[k] = temp 
results['any_severe__36km_Rot_*_16_16_lrate_0.0001_spatial_dropout_0.01_i_1_filters_[32, 64, 128, 256]_size_[2, 1, 2, 1]_pool_[2, 2, 2, 2]_loss_binary_crossentropy_results.pkl']

[{'args': {'lrate': 0.0001,
   'loss': 'binary_crossentropy',
   'activation_conv': 'relu',
   'activation_out': 'sigmoid',
   'p_spatial_dropout': 0.01,
   'filters': [32, 64, 128, 256],
   'size': [2, 1, 2, 1],
   'pool': [2, 2, 2, 2],
   'shape': (16, 16),
   'rotation': 0,
   'target_column': 'any_severe__36km',
   'i': 1},
  'predict_val_eval': [0.13702833652496338,
   0.9495965838432312,
   0.03855020925402641,
   0.9002200961112976,
   0.35952597856521606,
   5996.0,
   35947.0,
   0.0766206681728363,
   335991.0,
   13746.0],
  'predict_test_eval': [0.14527441561222076,
   0.9506197571754456,
   0.039156146347522736,
   0.8780367374420166,
   0.349609375,
   15272.0,
   78785.0,
   0.07908225804567337,
   751068.0,
   27835.0],
  'predict_train_eval': [0.13730941712856293,
   0.9413238167762756,
   0.03998349979519844,
   0.9222058653831482,
   0.46733176708221436,
   25601.0,
   139701.0,
   0.08043839782476425,
   1390515.0,
   69783.0],
  'history': {'loss': [0.7436586618423

In [ ]:
metrics=[tf.keras.metrics.SparseCategoricalAccuracy(), tf.keras.metrics.MeanSquaredError(name='Brier score'),
    tf.keras.metrics.AUC(name='auc'), tf.keras.metrics.AUC(name='prc', curve='PR'),     
         tf.keras.metrics.FalseNegatives(thresholds=thresholds), tf.keras.metrics.FalsePositives(thresholds=thresholds),
         tf.keras.metrics.MeanAbsoluteError(), tf.keras.metrics.TrueNegatives(thresholds=thresholds),
         tf.keras.metrics.TruePositives(thresholds=thresholds)]

In [33]:
scores=['accuracy','mse','bs','auc','prc','fn','fp','mae','tn','tp']
#scores=[ 'mae','accuracy','prc','auc', 'mse' ... ,'bs','tn']

In [29]:
mean_scores={}
for i, scores in enumerate(scores):
    mean_scores[scores] = { k : np.mean(hp_results[k][i]) for k in hp_results.keys()}

In [34]:
for score in scores:
    print(score)

accuracy
mse
bs
auc
prc
fn
fp
mae
tn
tp


In [35]:
for score in scores:
    print(score)
    print(np.nanmax([mean_scores[score][k] for k in mean_scores[score].keys() ]))
    #print(np.mean(mean_scores[score].values()))
    #print(np.min(mean_scores[score].values()))

accuracy
0.2254979431629181
mse
0.9427213430404663
bs
0.054918496310710906
auc
0.9036044359207154
prc
0.40633881092071533
fn
23025.2
fp
62963.2
mae
0.12183634340763091
tn
380430.8
tp
18350.0
